In [1]:
from detectron2 import model_zoo
from detectron2.config import get_cfg
from resnet import build_delta_resnet_backbone
from detectron2.modeling import build_resnet_backbone
from detectron2.layers import ShapeSpec
import time, torch, os, cv2
from torch.utils.data import Dataset
from torchvision.io import read_image
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
from detectron2.utils.logger import setup_logger
setup_logger()
from torchsummary import summary

# import some common libraries
from  matplotlib import pyplot as plt

# import some common detectron2 utilities
from detectron2.engine import DefaultPredictor, DefaultTrainer
# from detectron2.utils.visualizer import Visualizer
# from detectron2.data import MetadataCatalog
from deltacnn import DCConv2d, DCBackend, DCThreshold
# from torch.utils.tensorboard import SummaryWriter
DCConv2d.backend = DCBackend.cudnn
os.environ['PYTORCH_NO_CUDA_MEMORY_CACHING'] = "1"
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [2]:
class CustomImageDataset(Dataset):
    def __init__(self, img_dir, images, transform=None, target_transform=None):
        self.img_dir = img_dir
        self.images = images
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.images[0])
        a=self.images[idx]
        # image = read_image(img_path)
        image = cv2.imread(img_path)
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image

In [3]:
img_dir = 'D:/Samsung/Delta-Detectron2/MVI_39051'
images = os.listdir(img_dir)[:30]
dataset = CustomImageDataset(img_dir, images)
dataloader = DataLoader(dataset, batch_size=1, shuffle=False)

In [8]:
torch.cuda.empty_cache()

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_C4_3x.yaml"))
# cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_C4_1x.yaml"))
cfg.MODEL.WEIGHTS = "D:/Samsung/Delta-Detectron2/faster-rcnn_resnet50.pkl"

cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
cfg.MODEL.DEVICE='cuda'

DCThreshold.t_default=3
torch.cuda.empty_cache()

In [9]:
cfg.MODEL.BACKBONE.NAME = 'build_resnet_backbone'
model = DefaultPredictor(cfg)
torch.save(model, 'faster-rcnn_resnet50.pth')
start = time.time()
with torch.cuda.amp.autocast():
    for img in tqdm(dataset):
        out = model(img)
print(time.time()-start)

  0%|          | 0/30 [00:00<?, ?it/s]

RuntimeError: CUDA error: out of memory

In [6]:
torch.cuda.empty_cache()
DCConv2d.backend = DCBackend.deltacnn
cfg.MODEL.BACKBONE.NAME = 'build_delta_resnet_backbone'
cfg.MODEL.WEIGHTS = "D:/Samsung/Delta-Detectron2/faster-rcnn_resnet50_channel_last.pkl"
model = DefaultPredictor(cfg)
start = time.time()
with torch.cuda.amp.autocast():
    for img in tqdm(dataset):
        out = model(img)
print(time.time()-start)

Some model parameters or buffers are not found in the checkpoint:
backbone.res2.0.conv1.norm.num_batches_tracked
backbone.res2.0.conv2.norm.num_batches_tracked
backbone.res2.0.conv3.norm.num_batches_tracked
backbone.res2.0.shortcut.norm.num_batches_tracked
backbone.res2.1.conv1.norm.num_batches_tracked
backbone.res2.1.conv2.norm.num_batches_tracked
backbone.res2.1.conv3.norm.num_batches_tracked
backbone.res2.2.conv1.norm.num_batches_tracked
backbone.res2.2.conv2.norm.num_batches_tracked
backbone.res2.2.conv3.norm.num_batches_tracked
backbone.res3.0.conv1.norm.num_batches_tracked
backbone.res3.0.conv2.norm.num_batches_tracked
backbone.res3.0.conv3.norm.num_batches_tracked
backbone.res3.0.shortcut.norm.num_batches_tracked
backbone.res3.1.conv1.norm.num_batches_tracked
backbone.res3.1.conv2.norm.num_batches_tracked
backbone.res3.1.conv3.norm.num_batches_tracked
backbone.res3.2.conv1.norm.num_batches_tracked
backbone.res3.2.conv2.norm.num_batches_tracked
backbone.res3.2.conv3.norm.num_batc

  0%|          | 0/30 [00:00<?, ?it/s]

RuntimeError: CUDA error: out of memory